#Metodo de K-Means para clasificar articulos de acuerdo a sus ventas

##Se importan las bibliotecas necesarias

In [63]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing

import plotly.express as px
import plotly.graph_objects as go

##Se carga la información por modelo con su venta reconstruida

In [6]:
ids=[908839,154201,166340,908834,823456,166338,823454,831878]
vta=[4359,2769,2547,4736,3489,2207,1551,2461]
df = pd.DataFrame(list(zip(ids, vta)),
               columns =['ART', 'VTA'])
df

,ART,VTA
0,908839,4359
1,154201,2769
2,166340,2547
3,908834,4736
4,823456,3489
5,166338,2207
6,823454,1551
7,831878,2461


In [7]:
df=pd.read_excel('pantalon.xlsx')
df

,ART,VTA
0,908839,4359
1,154201,2769
2,166340,2547
3,908834,4736
4,823456,3489
5,166338,2207
6,823454,1551
7,831878,2461


Se normaliza la clave del articulo para que no influya en la creacion de los grupos

In [8]:
x = df['ART'].values.reshape(-1, 1) #se convierte a vector
min_max_scaler = preprocessing.MinMaxScaler() #se escala
x_scaled = min_max_scaler.fit_transform(x)
df['art_norm'] = pd.DataFrame(x_scaled) #se agrega al dataframe
df

,ART,VTA,art_norm
0,908839,4359,1.000000
1,154201,2769,0.000000
2,166340,2547,0.016086
3,908834,4736,0.999993
4,823456,3489,0.886856
5,166338,2207,0.016083
6,823454,1551,0.886853
7,831878,2461,0.898016


In [9]:
X=df[['art_norm','VTA']].to_numpy()

##Se grafica la informacion sin agrupar

In [21]:
fig = px.scatter(df, x="art_norm", y="VTA",title="Ventas pantalon chino",
                  marginal_y="box")
fig.show()

##Se aplica el método K-Means con diferentes centroides

Se aplica el método de k-means desde 1 hasta 8 centroides para verificar su distorción con el método del codo. Esto nos ayuda a identificar el número de grupos que se requieren

In [40]:
distorcions=[]
for i in range(1,9):
    km=KMeans(n_clusters=i,
             init='k-means++',
             n_init=10,
             max_iter=300,
             random_state=0)
    km.fit(X)
    distorcions.append(km.inertia_)

In [41]:
datos={'Clusters':range(1,9),
       'Distorciones':distorcions}
dfcodo=pd.DataFrame(datos)
dfcodo

,Clusters,Distorciones
0,1,8.375470e+06
1,2,1.694310e+06
2,3,9.473614e+05
3,4,2.329411e+05
4,5,1.215797e+05
5,6,5.051519e+04
6,7,3.698389e+03
7,8,0.000000e+00


In [42]:
fig = px.line(dfcodo, x='Clusters', y='Distorciones',title='Metodo del codo')
fig.show()

##Se usa el número de centroides ideal

Se identifica que el numero de grupos debe ser 3 porque a mayor numero de grupos las distorciones ya no se mejoran. La variable y_km guarda las clasificaciones.

In [49]:
km=KMeans(n_clusters=3,
         init='k-means++',
         n_init=10,
         max_iter=300,
         tol=1e-4,
         random_state=0)
y_km=km.fit_predict(X)

Se grafican los grupos

In [62]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=X[y_km==0,0], y=X[y_km==0,1],
                    mode='markers',
                    name='Regular'))
fig.add_trace(go.Scatter(x=X[y_km==1,0], y=X[y_km==1,1],
                    mode='markers',
                    name='Riesgo'))
fig.add_trace(go.Scatter(x=X[y_km==2,0], y=X[y_km==2,1],
                    mode='markers',
                    name='Bueno'))
fig.add_trace(go.Scatter(x=km.cluster_centers_[:,0], y=km.cluster_centers_[:,1],
                    mode='markers',marker_size=km.cluster_centers_[:,1]/200,
                    name='Centroides'))
fig.update_layout(title='Regla de compra Pantalon chino')
fig.show()

##Se obtiene la regla de compra

Clasificación de regla de compra para el pantalón chino tomando los valores de los centroides

In [64]:
sorted(km.cluster_centers_[:,1])

[2307.0, 3489.0, 4547.5]